In [2]:
import ee
import geemap

In [23]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [26]:
site = ee.FeatureCollection('users/Introduction/study_area')
#Map.addLayer(site, {}, "study_area")

site_dist = ee.FeatureCollection('users/Introduction/study_dist')
Map.addLayer(site_dist, {}, "study_dist")

In [10]:
# Assignment 1

# Export the Night Lights images for May,2015 and May,2020
# Filter the collection to the date range
# Extract the 'avg_rad' band which represents the nighttime lights
# Clip the image to the geometry of your city
# Export the resulting image as a GeoTIFF file.

# Hint1:

# There are 2 VIIRS Nighttime Day/Night collections
# Use the one that corrects for stray light

# Hint2:

# The collection contains 1 global image per month
# After filtering for the month, there will be only 1 image in the collection

# You can use the following technique to extract that image
# image = ee.Image(filtered.first())

#==================================================================
#                 BEGINNING OF ASSIGNMENT 1 - MUKESH RAY
#==================================================================

# Loading and Filtering the study area from GAUL dataset -- in this case Delhi, India
delhi = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2") \
              .filter(ee.Filter.eq("ADM1_NAME", "Delhi"))



Map.centerObject(site, 9)
Map.addLayer(site)

# Loading the VIIRS image collection
viirs = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG")

# Filtering one year of VIIRS dataset for 2015 and taking the first image
night2015 = viirs.filter(ee.Filter.date("2015-01-01", "2015-12-31")) \
                      .select('avg_rad') \
                      .first()

# Filtering last one month of VIIRS dataset for 2020 (December)
night2020 = viirs.filter(ee.Filter.date("2020-12-01", "2020-12-31")) \
                      .select('avg_rad') \
                      .first()

# Visualizing the the datasets for 2015 and 2020
Map.addLayer(night2015.clip(site), {'min':0, 'max':50, 'palette': ['0c0c0c', '071aff', 'ff0000', 'ffbd03', 'fbff05', 'fffdfd']}, 'Night2015')
Map.addLayer(night2020.clip(site), {'min':0, 'max':50, 'palette': ['0c0c0c', '071aff', 'ff0000', 'ffbd03', 'fbff05', 'fffdfd']}, 'Night2020')

# Find the mean radiance in the Delhi for 2015 and 2020
mean2015 = night2015.reduceRegion(
  reducer= ee.Reducer.mean(),
  geometry = site,
  scale = 500,
  maxPixels = 1e9
  )

mean2020 = night2020.reduceRegion(
  reducer= ee.Reducer.mean(),
  geometry = site,
  scale = 500,
  maxPixels = 1e9
  )

print("Mean Radiance in 2015: ", mean2015.get('avg_rad').getInfo())
print("Mean Radiance in 2020: ", mean2020.get('avg_rad').getInfo())

num2015 = ee.Number(mean2015.get('avg_rad'))
num2020 = ee.Number(mean2020.get('avg_rad'))

print(f"New Mean2015 is: {num2015.getInfo():.2f}")

# By comparing the mean of night light in 2015 and 2020, it is observed that the night light has decreased in 2020
# compared to 2015.


# Export.image.toDrive({
#   image: night2020,
#   description: "night2020",
#   folder: "GEE_Workshop",
#   region: delhi,
#   scale: 500,
#   maxPixels: 1e10,
#   fileFormat: 'GeoTIFF'
#   })

# END OF ASSIGNMENT 1


Mean Radiance in 2015:  5.32589382353451
Mean Radiance in 2020:  6.551199360512761
New Mean2015 is: 5.33


In [9]:
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
filename = os.path.join(out_dir, 'nightlight_2015.tif')

In [10]:
geemap.ee_export_image(night2015, filename = filename, scale = 500, region= site.geometry(), file_per_band= False)

The filename must end with .tif


In [11]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt

In [21]:
img = night2015

aoi = site.geometry()

night_img = geemap.ee_to_numpy(img, region=aoi)

print(night_img.shape)

(294, 480, 1)
